In [1]:
#host - trusted-prediction-research.c8qtxwbvasgw.ap-south-1.rds.amazonaws.com
#user - postgres
#passoword - ynKrco0QWa56iez8sKtG

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
import numpy as np
import pandas as pd
import psycopg2

# Extracting IPL2020 Data

In [5]:
conn = psycopg2.connect("dbname=cricket host =sportsanalytics.in user=postgres password=Sumit@123 ")

In [6]:
series = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4 OR league_id = 3", conn)

In [7]:
IPL_player_battingstats = pd.read_sql_query("SELECT * FROM cds_playerstats_playerbattingstats WHERE league_id = 4  ", conn)
T20_player_battingstats = pd.read_sql_query("SELECT * FROM cds_playerstats_playerbattingstats WHERE league_id = 3  ", conn)
IPL_player_bowlingstats = pd.read_sql_query("SELECT * FROM cds_playerstats_playerbowlingstats WHERE league_id = 4  ", conn)
T20_player_bowlingstats = pd.read_sql_query("SELECT * FROM cds_playerstats_playerbowlingstats WHERE league_id = 3  ", conn)

In [8]:
playerid_order = pd.read_sql_query("SELECT * FROM cds_scorecard_matchplayers", conn)

In [9]:
playerid_names = pd.read_sql_query("SELECT id,name FROM cds_player_player", conn)

In [10]:
playerid_names = dict(zip(playerid_names['id'],playerid_names['name']))

In [11]:
bowlerid_type = pd.read_csv("bowlerid_type")

In [12]:
bowlerid_type = bowlerid_type.dropna()

# Data preprocessing

In [13]:
IPL_player_battingstats = IPL_player_battingstats.drop(['id','ranking','style','centuries','half_centuries','zero_outs','highest_run','span','league_id'],1)
T20_player_battingstats = T20_player_battingstats.drop(['id','ranking','style','centuries','half_centuries','zero_outs','highest_run','span','league_id'],1)
T20_player_bowlingstats = T20_player_bowlingstats.drop(['id','ranking','style','stumpings','bbl','catches','maidens','span','league_id'],1)
IPL_player_bowlingstats = IPL_player_bowlingstats.drop(['id','ranking','style','stumpings','bbl','catches','maidens','span','league_id'],1)                       

In [14]:
playerid_order = playerid_order[{'player_id','order'}]
playerid_order = dict(zip(playerid_order.groupby('player_id').mean().index,round(playerid_order.groupby('player_id').mean()['order'])))
plidorder = {}
for i in IPL_player_battingstats['player_id']:
    plidorder[i] = playerid_order[i]
playerid_order = plidorder

## Indices for player ranking

In [15]:
pid = IPL_player_battingstats['player_id']
hss = (IPL_player_battingstats['fours']*4 + IPL_player_battingstats['sixes']*6)/IPL_player_battingstats['balls_faced']
IPL_HSS = dict((zip(pid,hss)))
pid = T20_player_battingstats['player_id']
hss = (T20_player_battingstats['fours']*4 + T20_player_battingstats['sixes']*6)/T20_player_battingstats['balls_faced']
T20_HSS = dict((zip(pid,hss)))

In [16]:
pid = IPL_player_battingstats['player_id']
fs = IPL_player_battingstats['not_out']/IPL_player_battingstats['innings']
IPL_FS = dict((zip(pid,fs)))
pid = T20_player_battingstats['player_id']
fs = T20_player_battingstats['not_out']/IPL_player_battingstats['innings']
T20_FS = dict((zip(pid,fs)))

In [17]:
pid = IPL_player_battingstats['player_id']
sr = IPL_player_battingstats['strike_rate']
IPL_SR = dict((zip(pid,sr)))
pid = T20_player_battingstats['player_id']
sr = T20_player_battingstats['strike_rate']
T20_SR = dict((zip(pid,sr)))

In [18]:
pid = IPL_player_battingstats['player_id']
average = IPL_player_battingstats['average']
IPL_CS = dict((zip(pid,average)))
pid = T20_player_battingstats['player_id']
average = T20_player_battingstats['average']
T20_CS = dict((zip(pid,average)))

In [19]:
pid = IPL_player_battingstats['player_id']
RBW = (IPL_player_battingstats['runs'] - (IPL_player_battingstats['fours']*4 + IPL_player_battingstats['sixes']*6))/(IPL_player_battingstats['balls_faced'] - IPL_player_battingstats['fours'] - IPL_player_battingstats['sixes'])
IPL_RBW = dict((zip(pid,RBW)))
pid = T20_player_battingstats['player_id']
RBW = (T20_player_battingstats['runs'] - (T20_player_battingstats['fours']*4 + T20_player_battingstats['sixes']*6))/(T20_player_battingstats['balls_faced'] - T20_player_battingstats['fours'] - T20_player_battingstats['sixes'])
T20_RBW = dict((zip(pid,RBW)))

In [20]:
pid = IPL_player_bowlingstats['player_id']
es = IPL_player_bowlingstats['economy']
IPL_ES = dict((zip(pid,es)))
pid = T20_player_bowlingstats['player_id']
es = T20_player_bowlingstats['economy']
T20_ES = dict((zip(pid,es)))

In [21]:
pid = IPL_player_bowlingstats['player_id']
cs = IPL_player_bowlingstats['runs']/IPL_player_bowlingstats['wickets']
IPL_BCS = dict((zip(pid,cs)))
pid = T20_player_bowlingstats['player_id']
cs = T20_player_bowlingstats['runs']/IPL_player_bowlingstats['wickets']
T20_BCS = dict((zip(pid,cs)))

In [22]:
pid = IPL_player_bowlingstats['player_id']
wts = IPL_player_bowlingstats['balls']/IPL_player_bowlingstats['wickets']
IPL_WTS = dict((zip(pid,wts)))
pid = T20_player_bowlingstats['player_id']
wts = T20_player_bowlingstats['balls']/IPL_player_bowlingstats['wickets']
T20_WTS = dict((zip(pid,wts)))

In [23]:
pid = IPL_player_bowlingstats['player_id']
bwts = (IPL_player_bowlingstats['wicket_4']+IPL_player_bowlingstats['wicket_5'])/IPL_player_bowlingstats['innings']
IPL_BWTS = dict((zip(pid,bwts)))
pid = T20_player_bowlingstats['player_id']
bwts = (T20_player_bowlingstats['wicket_4']+T20_player_bowlingstats['wicket_5'])/T20_player_bowlingstats['innings']
T20_BWTS = dict((zip(pid,bwts)))

In [24]:
pid = IPL_player_bowlingstats['player_id']
sps = (IPL_player_bowlingstats['wickets']-4*IPL_player_bowlingstats['wicket_4']-5*IPL_player_bowlingstats['wicket_5'])/(IPL_player_bowlingstats['innings']-IPL_player_bowlingstats['wicket_4']-IPL_player_bowlingstats['wicket_5'])
IPL_SPS = dict((zip(pid,sps)))
pid = T20_player_bowlingstats['player_id']
sps = (T20_player_bowlingstats['wickets']-4*T20_player_bowlingstats['wicket_4']-5*T20_player_bowlingstats['wicket_5'])/(T20_player_bowlingstats['innings']-T20_player_bowlingstats['wicket_4']-T20_player_bowlingstats['wicket_5'])
T20_SPS = dict((zip(pid,sps)))

# Ranking algorithm for Batsmen

In [25]:
import operator
finishers = (list(dict( sorted(IPL_FS.items(), key=operator.itemgetter(1),reverse=True)).keys())[0:150])   #taking 150 finishers

In [26]:
top_order_batsmen = list(dict( sorted(playerid_order.items(), key=operator.itemgetter(1))).keys())[0:150]

In [27]:
allrounderid = pd.read_csv('Allrounderid.csv')
allrounderid = list(allrounderid['0'])

In [28]:
middle_order_batsmen = list(dict( sorted(playerid_order.items(), key=operator.itemgetter(1))).keys())[151:]

In [29]:
inexp_batsman_id = list(IPL_player_battingstats['player_id'][IPL_player_battingstats['innings']<6])

In [30]:
for i in inexp_batsman_id :
    if(top_order_batsmen.count(i)==1):
        top_order_batsmen.remove(i)
    if(allrounderid.count(i)==1):
        allrounderid.remove(i)
    if(middle_order_batsmen.count(i)==1):
        middle_order_batsmen.remove(i)
for i in allrounderid:
    if(top_order_batsmen.count(i)==1):
        top_order_batsmen.remove(i)
    if(middle_order_batsmen.count(i)==1):
        middle_order_batsmen.remove(i)

In [31]:
tob_ranking_points = {}
for i in top_order_batsmen:
    if(list(T20_CS.keys()).count(i)==0):
        tob_ranking_points[i]  = 0.383*IPL_RBW[i]+0.247*IPL_CS[i]+0.196*IPL_CS[i]+0.174*IPL_FS[i]
    else:
        tob_ranking_points[i]  = 0.383*IPL_RBW[i]+0.247*T20_CS[i]+0.196*IPL_CS[i]+0.174*T20_FS[i]
tob_ranks = (list(dict( sorted(tob_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [32]:
mob_ranking_points = {}
for i in middle_order_batsmen:
    if(list(T20_CS.keys()).count(i)==0):
        mob_ranking_points[i]  = 0.339*IPL_CS[i]+0.337*IPL_HSS[i]+0.234*IPL_FS[i]+0.082*IPL_FS[i]+0.008*IPL_FS[i]
    else:
        mob_ranking_points[i]  = 0.339*IPL_CS[i]+0.337*IPL_HSS[i]+0.234*IPL_FS[i]+0.082*T20_FS[i]+0.008*IPL_FS[i]
mob_ranks = (list(dict( sorted(mob_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [33]:
finishers_ranking_points = {}
for i in finishers:
    if(list(T20_CS.keys()).count(i)==0):
        finishers_ranking_points[i]  = 0.299*IPL_FS[i]+0.262*IPL_HSS[i]+0.241*IPL_FS[i]+0.108*IPL_HSS[i]+0.089*IPL_RBW[i]
    else:
        finishers_ranking_points[i]  = 0.299*IPL_FS[i]+0.262*T20_HSS[i]+0.241*T20_FS[i]+0.108*IPL_HSS[i]+0.089*IPL_RBW[i]
finishers_ranks = (list(dict( sorted(finishers_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [34]:
inexp_ranking_points = {}
for i in inexp_batsman_id:
    if(list(T20_CS.keys()).count(i)==0):
        inexp_ranking_points[i]  = 0.299*IPL_FS[i]+0.262*IPL_HSS[i]+0.241*IPL_FS[i]+0.108*IPL_HSS[i]+0.089*IPL_RBW[i]
    else:
        inexp_ranking_points[i]  = 0.364*T20_CS[i]+0.301*T20_HSS[i]+0.113*T20_FS[i]+0.107*T20_FS[i]+0.105*T20_RBW[i]
inexp_ranks = tob_ranks = (list(dict( sorted(inexp_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

# Ranking Algorithm for bowlers and all rounders

In [39]:
bowlerid_type = dict(zip(bowlerid_type['id'],bowlerid_type['bowler_type']))

In [40]:
IPL_bowlerid_type = {}
IPL_bowler_nodata = []

In [41]:
for i in IPL_player_bowlingstats['player_id']:
    if(bowlerid_type.get(i)):
        IPL_bowlerid_type[i] = bowlerid_type[i]
    else:
        IPL_bowler_nodata.append(i)

In [42]:
IPL_bowlerid_type[1062] = 'PACER'   #manually adding bowler type whose data is not available using nodata list
IPL_bowlerid_type[42] = 'PACER'

In [43]:
bowlerids = list(IPL_bowlerid_type.keys())

In [44]:
for i in allrounderid:
    if(bowlerids.count(i)==1):
        bowlerids.remove(i)

In [45]:
inexp_bowlers_id = list(IPL_player_bowlingstats['player_id'][IPL_player_bowlingstats['innings']<6])

In [46]:
for i in inexp_bowlers_id:
    if(allrounderid.count(i)==1):
        allrounderid.remove(i)
    if(bowlerids.count(i)==1):
        bowlerids.remove(i)

In [47]:
bowlerspinner = []
bowlerpacer = []
allrounderspinner = []
allrounderpacer = []
for i in allrounderid:
    if(bowlerid_type[i]=='PACER'):
        allrounderpacer.append(i)
    else:
        allrounderspinner.append(i)
bowlerids.remove(42)
for i in bowlerids:
    if(bowlerid_type[i]=='PACER'):
        bowlerpacer.append(i)
    else:
        bowlerspinner.append(i)
    

In [48]:
spinb_ranking_points = {}
for i in bowlerspinner:
    if(list(T20_BCS.keys()).count(i)==0):
        spinb_ranking_points[i]  = 0.490*IPL_SPS[i]+0.157*IPL_SPS[i]+0.157*IPL_WTS[i]+0.102*IPL_BWTS[i]
    else:
        spinb_ranking_points[i]  = 0.490*IPL_SPS[i]+0.157*T20_SPS[i]+0.157*IPL_WTS[i]+0.102*T20_BWTS[i]
spinb_ranks = (list(dict( sorted(spinb_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [49]:
paceb_ranking_points = {}
for i in bowlerpacer:
    if(list(T20_BCS.keys()).count(i)==0):
        paceb_ranking_points[i]  = 0.197*IPL_BWTS[i]+0.299*IPL_SPS[i]+0.328*IPL_ES[i]+0.175*IPL_SPS[i]
    else:
        paceb_ranking_points[i]  = 0.197*IPL_BWTS[i]+0.299*IPL_SPS[i]+0.328*IPL_ES[i]+0.175*T20_SPS[i]
paceb_ranks = (list(dict( sorted(paceb_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [50]:
inexpb_ranking_points = {}
for i in inexp_bowlers_id:
    if(list(T20_BCS.keys()).count(i)==0):
        inexpb_ranking_points[i]  = 0.829*IPL_SPS[i]+0.120*IPL_BCS[i]+0.050*IPL_ES[i]
    else:
        inexpb_ranking_points[i]  = 0.829*IPL_SPS[i]+0.120*T20_BCS[i]+0.050*T20_ES[i]
inexpb_ranks = (list(dict( sorted(inexpb_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [51]:
spinall_ranking_points = {}
allrounderspinner.remove(518)
for i in allrounderspinner:
    if(list(T20_BCS.keys()).count(i)==0):
        spinall_ranking_points[i]  = 0.393*IPL_BWTS[i]+0.250*IPL_ES[i]+0.222*IPL_ES[i]+0.135*IPL_WTS[i]
    else:
        spinall_ranking_points[i]  = 0.393*T20_BWTS[i]+0.250*IPL_ES[i]+0.222*T20_ES[i]+0.135*T20_WTS[i]
spinall_ranks = (list(dict( sorted(spinall_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [52]:
paceall_ranking_points = {}
for i in allrounderpacer:
    if(list(T20_BCS.keys()).count(i)==0):
        paceall_ranking_points[i]  = 0.320*IPL_SPS[i]+0.222*IPL_WTS[i]+0.190*IPL_BWTS[i]+0.173*IPL_BCS[i]+0.095*IPL_SPS[i]
    else:
        paceall_ranking_points[i]  = 0.320*T20_SPS[i]+0.222*T20_WTS[i]+0.190*T20_BWTS[i]+0.173*T20_BCS[i]+0.095*IPL_SPS[i]
paceall_ranks = (list(dict( sorted(paceall_ranking_points.items(), key=operator.itemgetter(1),reverse=True)).keys()))

In [58]:
#playing_11 = [23, 18, 17, 19, 1063, 1, 132, 150, 608, 676, 31, 589, 1030, 29, 86, 1061, 200, 1062, 599, 274, 1007, 1044]

In [59]:
# Selecting 3 top order batsmen
tob_team = []
for i in tob_ranks :
    if(playing_11.count(i)==1):
        tob_team.append(i)
    if(len(np.unique(tob_team)==3)):
        break
# Selecting 3 middle order batsmen
mob_team = []
for i in mob_ranks :
    if(playing_11.count(i)==1):
        mob_team.append(i)
    if(len(np.unique(mob_team))==3):
        break
# Selecting 2 best finisher
finisher_team = []
for i in finishers_ranks :
    if(playing_11.count(i)==1):
        finisher_team.append(i)
    if(len(np.unique(finisher_team))==2):
        break
# Selecting 2 spin allrounder
spinall_team = []
for i in spinall_ranks :
    if(playing_11.count(i)==1):
        spinall_team.append(i)
    if(len(np.unique(spinall_team))==2):
        break
# Selecting 2 pace allrounder
paceall_team = []
for i in paceall_ranks :
    if(playing_11.count(i)==1):
        paceall_team.append(i)
    if(len(np.unique(paceall_team))==2):
        break
# Selecting 2 spinner 
spinb_team = []
for i in spinb_ranks :
    if(playing_11.count(i)==1):
        spinb_team.append(i)
    if(len(np.unique(spinb_team))==3):
        break
# Selecting 3 pace bowler
paceb_team = []
for i in paceb_ranks :
    if(playing_11.count(i)==1):
        paceb_team.append(i)
    if(len(np.unique(paceb_team))==3):
        break
# Selecting 2 inexperienced player
inexp = []
for i in inexp_ranks :
    if(playing_11.count(i)==1):
        inexp.append(i)
        break
for i in inexpb_ranks :
    if(playing_11.count(i)==1):
        inexp.append(i)
        break
import random
pred_teams = []
no_teams = 10
for i in range(0,no_teams):
    team = []
    if(len(tob_team)>=2) : team.append(random.sample(tob_team,2))
    else : team.append([tob_team[0]])
    if(len(mob_team)>=2) : team.append(random.sample(mob_team,2))
    else : team.append([mob_team[0]])
    if(len(finisher_team)>=1): team.append(random.sample(finisher_team,1))
    if(len(spinall_team)>=1): team.append(random.sample(spinall_team,1))
    if(len(paceall_team)>=1): team.append(random.sample(paceall_team,1))
    if(len(spinb_team)>=1): team.append(random.sample(spinb_team,1))
    if(len(paceb_team)>=2): team.append(random.sample(paceb_team,2))
    if(len(inexp)>=1): team.append(random.sample(inexp,1))
    team = [item for sublist in team for item in sublist]
    while(len(np.unique(team))!=11):
        team.append(random.choice(playing_11))
    team = np.unique(team)
    pred_teams.append(team)